In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Quick tour

Get up and running with 🤗 Transformers! Start using the [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) for rapid inference, and quickly load a pretrained model and tokenizer with an [AutoClass](https://huggingface.co/docs/transformers/master/en/./model_doc/auto) to solve your text, vision or audio task.

<Tip>

All code examples presented in the documentation have a toggle on the top left for PyTorch and TensorFlow. If
not, the code is expected to work for both backends without any change.

</Tip>

## Pipeline

[pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) is the easiest way to use a pretrained model for a given task.

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/tiZFewofSLM?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

The [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) supports many common tasks out-of-the-box:

**Text**:
* Sentiment analysis: classify the polarity of a given text.
* Text generation (in English): generate text from a given input.
* Name entity recognition (NER): label each word with the entity it represents (person, date, location, etc.).
* Question answering: extract the answer from the context, given some context and a question.
* Fill-mask: fill in the blank given a text with masked words.
* Summarization: generate a summary of a long sequence of text or document.
* Translation: translate text into another language.
* Feature extraction: create a tensor representation of the text.

**Image**:
* Image classification: classify an image.
* Image segmentation: classify every pixel in an image.
* Object detection: detect objects within an image.

**Audio**:
* Audio classification: assign a label to a given segment of audio.
* Automatic speech recognition (ASR): transcribe audio data into text.

<Tip>

For more details about the [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) and associated tasks, refer to the documentation [here](https://huggingface.co/docs/transformers/master/en/./main_classes/pipelines).

</Tip>

### Pipeline usage

In the following example, you will use the [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) for sentiment analysis.

Install the following dependencies if you haven't already:

```bash
pip install torch
```

Import [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) and specify the task you want to complete:

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

The pipeline downloads and caches a default [pretrained model](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) and tokenizer for sentiment analysis. Now you can use the `classifier` on your target text:

In [ ]:
classifier("We are very happy to show you the 🤗 Transformers library.")

[{"label": "POSITIVE", "score": 0.9998}]

For more than one sentence, pass a list of sentences to the [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) which returns a list of dictionaries:

In [ ]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309

The [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) can also iterate over an entire dataset. Start by installing the [🤗 Datasets](https://huggingface.co/docs/datasets/) library:

```bash
pip install datasets 
```

Create a [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) with the task you want to solve for and the model you want to use. Set the `device` parameter to `0` to place the tensors on a CUDA device:

In [ ]:
from transformers import pipeline

speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h", device=0)

Next, load a dataset (see the 🤗 Datasets [Quick Start](https://huggingface.co/docs/datasets/quickstart.html) for more details) you'd like to iterate over. For example, let's load the [SUPERB](https://huggingface.co/datasets/superb) dataset:

In [ ]:
import datasets

dataset = datasets.load_dataset("superb", name="asr", split="test")

Now you can iterate over the dataset with the pipeline. `KeyDataset` retrieves the item in the dictionary returned by the dataset:

In [ ]:
from transformers.pipelines.base import KeyDataset
from tqdm.auto import tqdm

for out in tqdm(speech_recognizer(KeyDataset(dataset, "file"))):
    print(out)

{"text": "HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOWER FAT AND SAUCE"}

### Use another model and tokenizer in the pipeline

The [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) can accommodate any model from the [Model Hub](https://huggingface.co/models), making it easy to adapt the [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline) for other use-cases. For example, if you'd like a model capable of handling French text, use the tags on the Model Hub to filter for an appropriate model. The top filtered result returns a multilingual [BERT model](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment) fine-tuned for sentiment analysis. Great, let's use this model!

In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

Use the [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModelForSequenceClassification) and ['AutoTokenizer'] to load the pretrained model and it's associated tokenizer (more on an `AutoClass` below):

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Then you can specify the model and tokenizer in the [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline), and apply the `classifier` on your target text:

In [ ]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier("Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.")

[{"label": "5 stars", "score": 0.7272651791572571}]

If you can't find a model for your use-case, you will need to fine-tune a pretrained model on your data. Take a look at our [fine-tuning tutorial](https://huggingface.co/docs/transformers/master/en/./training) to learn how. Finally, after you've fine-tuned your pretrained model, please consider sharing it (see tutorial [here](https://huggingface.co/docs/transformers/master/en/./model_sharing)) with the community on the Model Hub to democratize NLP for everyone! 🤗

## AutoClass

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/AhChOFRegn4?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

Under the hood, the [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModelForSequenceClassification) and [AutoTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoTokenizer) classes work together to power the [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline). An [AutoClass](https://huggingface.co/docs/transformers/master/en/./model_doc/auto) is a shortcut that automatically retrieves the architecture of a pretrained model from it's name or path. You only need to select the appropriate `AutoClass` for your task and it's associated tokenizer with [AutoTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoTokenizer). 

Let's return to our example and see how you can use the `AutoClass` to replicate the results of the [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline).

### AutoTokenizer

A tokenizer is responsible for preprocessing text into a format that is understandable to the model. First, the tokenizer will split the text into words called *tokens*. There are multiple rules that govern the tokenization process, including how to split a word and at what level (learn more about tokenization [here](https://huggingface.co/docs/transformers/master/en/./tokenizer_summary)). The most important thing to remember though is you need to instantiate the tokenizer with the same model name to ensure you're using the same tokenization rules a model was pretrained with.

Load a tokenizer with [AutoTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoTokenizer):

In [ ]:
from transformers import AutoTokenizer

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Next, the tokenizer converts the tokens into numbers in order to construct a tensor as input to the model. This is known as the model's *vocabulary*.

Pass your text to the tokenizer:

In [ ]:
encoding = tokenizer("We are very happy to show you the 🤗 Transformers library.")
print(encoding)

{"input_ids": [101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102],
 "attention_mask": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

The tokenizer will return a dictionary containing:

* [input_ids](https://huggingface.co/docs/transformers/master/en/./glossary#input-ids): numerical representions of your tokens.
* [atttention_mask](https://huggingface.co/docs/transformers/master/en/.glossary#attention-mask): indicates which tokens should be attended to.

Just like the [pipeline()](https://huggingface.co/docs/transformers/master/en/main_classes/pipelines#transformers.pipeline), the tokenizer will accept a list of inputs. In addition, the tokenizer can also pad and truncate the text to return a batch with uniform length:

In [ ]:
pt_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",
)

Read the [preprocessing](https://huggingface.co/docs/transformers/master/en/./preprocessing) tutorial for more details about tokenization.

### AutoModel

🤗 Transformers provides a simple and unified way to load pretrained instances. This means you can load an [AutoModel](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModel) like you would load an [AutoTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoTokenizer). The only difference is selecting the correct [AutoModel](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModel) for the task. Since you are doing text - or sequence - classification, load [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModelForSequenceClassification). The TensorFlow equivalent is simply [TFAutoModelForSequenceClassification](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.TFAutoModelForSequenceClassification):

In [ ]:
from transformers import AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

<Tip>

See the [task summary](https://huggingface.co/docs/transformers/master/en/./task_summary) for which [AutoModel](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModel) class to use for which task.

</Tip>

Now you can pass your preprocessed batch of inputs directly to the model. If you are using a PyTorch model, unpack the dictionary by adding `**`. For TensorFlow models, pass the dictionary keys directly to the tensors:

In [ ]:
pt_outputs = pt_model(**pt_batch)

The model outputs the final activations in the `logits` attribute. Apply the softmax function to the `logits` to retrieve the probabilities:

In [ ]:
from torch import nn

pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
print(pt_predictions)

tensor([[2.2043e-04, 9.9978e-01],
        [5.3086e-01, 4.6914e-01]], grad_fn=<SoftmaxBackward>)

<Tip>

All 🤗 Transformers models (PyTorch or TensorFlow) outputs the tensors *before* the final activation
function (like softmax) because the final activation function is often fused with the loss.

</Tip>

Models are a standard [`torch.nn.Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) or a [`tf.keras.Model`](https://www.tensorflow.org/api_docs/python/tf/keras/Model) so you can use them in your usual training loop. However, to make things easier, 🤗 Transformers provides a [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) class for PyTorch that adds functionality for distributed training, mixed precision, and more. For TensorFlow, you can use the `fit` method from [Keras](https://keras.io/). Refer to the [training tutorial](https://huggingface.co/docs/transformers/master/en/./training) for more details.

<Tip>

🤗 Transformers model outputs are special dataclasses so their attributes are autocompleted in an IDE.
The model outputs also behave like a tuple or a dictionary (e.g., you can index with an integer, a slice or a string) in which case the attributes that are `None` are ignored.

</Tip>

### Save a model

Once your model is fine-tuned, you can save it with its tokenizer using [PreTrainedModel.save_pretrained()](https://huggingface.co/docs/transformers/master/en/main_classes/model#transformers.PreTrainedModel.save_pretrained):

In [ ]:
pt_save_directory = "./pt_save_pretrained"
tokenizer.save_pretrained(pt_save_directory)
pt_model.save_pretrained(pt_save_directory)

When you are ready to use the model again, reload it with [PreTrainedModel.from_pretrained()](https://huggingface.co/docs/transformers/master/en/main_classes/model#transformers.PreTrainedModel.from_pretrained):

In [ ]:
pt_model = AutoModelForSequenceClassification.from_pretrained("./pt_save_pretrained")

One particularly cool 🤗 Transformers feature is the ability to save a model and reload it as either a PyTorch or TensorFlow model. The `from_pt` or `from_tf` parameter can convert the model from one framework to the other:

In [ ]:
from transformers import AutoModel

tokenizer = AutoTokenizer.from_pretrained(tf_save_directory)
pt_model = AutoModelForSequenceClassification.from_pretrained(tf_save_directory, from_tf=True)